In [1]:
import time
import numpy as np
import pandas as pd

import requests

In [2]:
df = pd.read_excel(r'.\datasets\250716.xlsx')
print(df.head())

   순번                   출발지 주소              중간집결지 주소            최종 도착지 주소  \
0   1  경기도 고양시 덕양구 중앙로633번길 12   경기도 고양시 덕양구 혜음로 142  경기도 고양시 덕양구 혜음로 142   
1   2   대구광역시 서구 북비산로61길 20-24      경상북도 경주시 태종로 516  경기도 고양시 덕양구 동헌로 305   
2   3   서울특별시 양천구 목동중앙본로11길 19  서울특별시 서초구 사평대로45길 26  강원특별자치도 동해시 대동로 430   
3   4        서울특별시 강남구 학동로 432       경기도 과천시 중앙로 294      경기도 과천시 중앙로 294   
4   5         대구광역시 남구 명덕로 236      대구광역시 수성구 무열로 56     대구광역시 수성구 무열로 56   

    중간집결 기한    최종도착 기한 이동방법  
0  0+1일 14시   0+1일 14시   개별  
1  0+28일 9시  0+28일 14시   집단  
2   0+3일 8시   0+3일 14시   집단  
3  0+1일 14시   0+1일 14시   개별  
4  0+3일 14시   0+3일 14시   개별  


In [3]:
num = np.array(df['순번'])

start = np.array(df['출발지 주소'])
mid = np.array(df['중간집결지 주소'])
end = np.array(df['최종 도착지 주소'])

In [4]:
def geocoding(address: list):
    url = "https://maps.apigw.ntruss.com/map-geocode/v2/geocode"
    
    headers = {
        "x-ncp-apigw-api-key-id": "oa77i9oz1h",
        "x-ncp-apigw-api-key": "xvC9h8wAZLXjsokASCSKjLfNJ5uR63sKBGz705KA",
        "Accept": "application/json"
    }

    params = {"query": address}
    resp = requests.get(url, headers = headers, params = params)
    resp.raise_for_status()
    data = resp.json()

    if data.get('addresses'):
        lat = float(data['addresses'][0]['y']) # 위도
        lng = float(data['addresses'][0]['x']) # 경도  
        return lat, lng
    
    else:
        lat = None
        lng = None
        return lat, lng

In [10]:
# geocoding
if __name__ == "__main__":
    chunk = 1_000
    start_index = 350_000
    last_index = 400_000
    
    for i in range(start_index, last_index, chunk):
        slc = slice(i, min(i + chunk, last_index))
        print(f'Processing records {slc.start} to {slc.stop}...')

        t0 = time.time()
        start_lat, start_lng = zip(*(geocoding(addr) for addr in start[slc]))
        mid_lat, mid_lng = zip(*(geocoding(addr) for addr in mid[slc]))
        end_lat, end_lng = zip(*(geocoding(addr) for addr in end[slc]))
        t1 = time.time()

        print(f'걸린시간(분): {(t1-t0)/60:.2f}')

        df_chunk = pd.DataFrame({'num': num[slc],
                       'start_lat': start_lat,
                       'start_lng': start_lng,
                       'mid_lat': mid_lat,
                       'mid_lng': mid_lng,
                       'end_lat': end_lat,
                       'end_lng': end_lng})
        
        mode = 'w' if i == start_index else 'a'
        header = True if i == start_index else False

        df_chunk.to_csv(r'.\results\transformed.csv', mode = mode, header = header, index = False)
        print(f'{slc.start} to {slc.stop} 저장 완료')

Processing records 350000 to 351000...
걸린시간(분): 4.74
350000 to 351000 저장 완료
Processing records 351000 to 352000...
걸린시간(분): 5.19
351000 to 352000 저장 완료
Processing records 352000 to 353000...
걸린시간(분): 4.50
352000 to 353000 저장 완료
Processing records 353000 to 354000...
걸린시간(분): 4.63
353000 to 354000 저장 완료
Processing records 354000 to 355000...
걸린시간(분): 4.49
354000 to 355000 저장 완료
Processing records 355000 to 356000...
걸린시간(분): 4.50
355000 to 356000 저장 완료
Processing records 356000 to 357000...
걸린시간(분): 4.95
356000 to 357000 저장 완료
Processing records 357000 to 358000...
걸린시간(분): 4.65
357000 to 358000 저장 완료
Processing records 358000 to 359000...
걸린시간(분): 5.11
358000 to 359000 저장 완료
Processing records 359000 to 360000...
걸린시간(분): 4.58
359000 to 360000 저장 완료
Processing records 360000 to 361000...
걸린시간(분): 4.52
360000 to 361000 저장 완료
Processing records 361000 to 362000...
걸린시간(분): 4.82
361000 to 362000 저장 완료
Processing records 362000 to 363000...
걸린시간(분): 4.50
362000 to 363000 저장 완료
Processing r